In [1]:
import re
from collections import defaultdict
import pickle
from random import choice

In [2]:
def gen_lines_from_files(list_of_corpuses):
    for corpus in list_of_corpuses:  
        with open(corpus, 'r', encoding='utf-8') as file:
            for line in file:
                yield line.lower()
                

def gen_tokens(lines):
    alphabet = re.compile(u'[а-яА-Яa-zA-Z0-9’]+')
    for line in lines:
        for token in alphabet.findall(line):
            yield token

            
def gen_trigrams(tokens):
    t0, t1 = next(tokens), next(tokens)
    for t2 in tokens:
        yield t0, t1, t2
        t0, t1 = t1, t2

        
def fit(list_of_corpuses):
    lines = gen_lines_from_files(list_of_corpuses)
    tokens = gen_tokens(lines)
    trigrams = gen_trigrams(tokens)
    bi, tri = defaultdict(int) , defaultdict(int) 
    for t0, t1, t2 in trigrams:
        bi[t0, t1] += 1
        tri[t0, t1, t2] += 1
    model = {}
    for (t0, t1, t2), freq in tri.items():
        if (t0, t1) in model:
            model[t0, t1].append((t2, freq/bi[t0, t1]))
        else:
            model[t0, t1] = [(t2, freq/bi[t0, t1])]  
    return model


def generate(model, length, prefix='-1'):
    phrase = ''
    bigrams = tuple(filter(lambda key: key[0] == prefix, model))
    #Если ввели 2 слова
    if tuple(prefix.split()) in model:
        t0, t1 = prefix.split()      
    # Если ввели одно слово и оно есть в биграммах
    elif prefix != '-1' and len(bigrams) != 0:
        t0, t1 = choice(tuple(filter(lambda key: key[0] == prefix, model)))
    # Введеного слова нет в биграммах или ничего не ввели
    else:
        t0, t1 = choice(tuple(model))
    phrase += t0
    for i in range(length-len(prefix.split())):
        phrase += ' ' + t1
        t0, t1 = t1, choice(model[t0, t1])[0]
    return phrase.capitalize() + '.'


In [3]:
model_ru = fit(['data_ru/Disgardium_5.txt', 'data_ru/Disgardium_1.txt'])

In [4]:
model_ru

{('дисгардиум', '5'): [('священная', 1.0)],
 ('5', 'священная'): [('война', 1.0)],
 ('священная', 'война'): [('данияр', 0.2),
  ('краткое', 0.2),
  ('официально', 0.2),
  ('начата', 0.2),
  ('закончена', 0.2)],
 ('война', 'данияр'): [('саматович', 1.0)],
 ('данияр', 'саматович'): [('сугралинов', 1.0)],
 ('саматович', 'сугралинов'): [('litrpgдисгардиум', 1.0)],
 ('сугралинов', 'litrpgдисгардиум'): [('5', 0.5), ('1', 0.5)],
 ('litrpgдисгардиум', '5'): [('под', 1.0)],
 ('5', 'под'): [('защитой', 1.0)],
 ('под', 'защитой'): [('нергала', 0.5), ('краша', 0.5)],
 ('защитой', 'нергала'): [('лучезарного', 1.0)],
 ('нергала', 'лучезарного'): [('армии', 0.047619047619047616),
  ('личность', 0.047619047619047616),
  ('который', 0.047619047619047616),
  ('во', 0.047619047619047616),
  ('уже', 0.047619047619047616),
  ('воздух', 0.047619047619047616),
  ('сплюнув', 0.047619047619047616),
  ('так', 0.047619047619047616),
  ('а', 0.047619047619047616),
  ('подвал', 0.047619047619047616),
  ('и', 0.047

In [5]:
# 1 префекс
print(generate(model=model_ru, length=30, prefix='ученые'))
print()
# 2 префекса
print(generate(model=model_ru, length=40, prefix='закричал он'))
print()
# Без префиксов
print(generate(model=model_ru, length=50))

Ученые предсказывают крах системы гражданства и кризис продовольствия более того теперь полностью пропал смысл вообще что либо ему отдавать не захотел куда удобнее было разбираться с мобами тысячного уровня они.

Закричал он вскакивая с лавочки у таверны он замахнулся мечом глава 17 боги и зверобоги йеми устрой мне встречу с апопом белым змеем мы идем долго не реагировал на низкий рокот там тамов разносящийся по джунглям от точки воскрешения.

Зелий здоровья и отправить в мир духов погибших матросов стало первым сюрпризом для ниндзя лутеров стало явление великого и ужасного краша моего алмазного червя магвай что то по проекту погруженные в совместное виртуальное рабочее пространство эйты в режиме энергосбережения изображал кота на подоконнике нежась в лучах солнца и подзаряжая аккумуляторы.


In [6]:
model_en = fit(['data_en/Harry_Potter_and_the_Sorcerers_Stone.txt'])

In [7]:
model_en

{('harry', 'potter'): [('and', 0.037037037037037035),
  ('voldemort’s', 0.037037037037037035),
  ('come', 0.037037037037037035),
  ('day', 0.037037037037037035),
  ('rolled', 0.037037037037037035),
  ('the', 0.1111111111111111),
  ('was', 0.037037037037037035),
  ('in', 0.037037037037037035),
  ('not', 0.07407407407407407),
  ('what', 0.037037037037037035),
  ('it', 0.07407407407407407),
  ('chorused', 0.037037037037037035),
  ('harry', 0.037037037037037035),
  ('ron', 0.037037037037037035),
  ('said', 0.1111111111111111),
  ('our', 0.037037037037037035),
  ('his', 0.037037037037037035),
  ('their', 0.037037037037037035),
  ('an’', 0.037037037037037035),
  ('do', 0.037037037037037035),
  ('she', 0.037037037037037035)],
 ('potter', 'and'): [('the', 0.5), ('what', 0.5)],
 ('and', 'the'): [('sorcerer’s', 0.012195121951219513),
  ('old', 0.012195121951219513),
  ('twinkling', 0.012195121951219513),
  ('racing', 0.012195121951219513),
  ('first', 0.012195121951219513),
  ('zoo', 0.012195121

In [8]:
# 1 префекс
print(generate(model=model_en, length=30, prefix='i'))
print()
# 2 префекса
print(generate(model=model_en, length=40, prefix='he knows'))
print()
# Без префиксов
print(generate(model=model_en, length=50))

I just thought hermione what should we write back tell them off to get expelled from he’ll flatten it or not oh right said hermione thoughtfully do you realize who.

He knows more or less everything that goes on here and there in an ordinary way he wanted at his broom no shouted hermione granger wearing a violet top hat fell off in the lead he’d done it ron.

Crash behind them and their younger sister began to eat before yer train leaves he said but it’s that stupid thing longbottom’s gran sent him letters hagrid would never betray dumbledore hagrid would never betray dumbledore hagrid would never ever forget tonight harry had to take sides so after that.


In [9]:
# Запись в файл
filename = 'trigram_model.pickle'
with open(filename, 'wb') as file:
    pickle.dump(model_en, file)

In [10]:
# Загрузка модели из файла
filename = 'trigram_model.pickle'
with open(filename, 'rb') as file:
    model_from_pickle = pickle.load(file)


In [11]:
model_from_pickle

{('harry', 'potter'): [('and', 0.037037037037037035),
  ('voldemort’s', 0.037037037037037035),
  ('come', 0.037037037037037035),
  ('day', 0.037037037037037035),
  ('rolled', 0.037037037037037035),
  ('the', 0.1111111111111111),
  ('was', 0.037037037037037035),
  ('in', 0.037037037037037035),
  ('not', 0.07407407407407407),
  ('what', 0.037037037037037035),
  ('it', 0.07407407407407407),
  ('chorused', 0.037037037037037035),
  ('harry', 0.037037037037037035),
  ('ron', 0.037037037037037035),
  ('said', 0.1111111111111111),
  ('our', 0.037037037037037035),
  ('his', 0.037037037037037035),
  ('their', 0.037037037037037035),
  ('an’', 0.037037037037037035),
  ('do', 0.037037037037037035),
  ('she', 0.037037037037037035)],
 ('potter', 'and'): [('the', 0.5), ('what', 0.5)],
 ('and', 'the'): [('sorcerer’s', 0.012195121951219513),
  ('old', 0.012195121951219513),
  ('twinkling', 0.012195121951219513),
  ('racing', 0.012195121951219513),
  ('first', 0.012195121951219513),
  ('zoo', 0.012195121

In [12]:
print(generate(model=model_en, length=50))

Know my youngest brother got past mcgonagall’s giant chess set by losing spectacularly to ron to unwrap his turban the few seconds’ silence the dog was guarding something she stood up sir professor dumbledore will be in slytherin all our food and you know quirrell already do you she said.


In [13]:
list_of_corpuses = ['C:\\Users\\home\\EXAM_TINKOFF\\new-project\\data_ru\\Disgardium_5.txt', 
                    'C:\\Users\\home\\EXAM_TINKOFF\\new-project\\data_ru\\Dyuma_A._Graf_Monte_Kristo.txt']

In [14]:
# С помощью класса

class MyTrigramModel:
    
    def __init__(self):
        pass
    
    @staticmethod
    def fit(list_of_corpuses):
        lines = gen_lines_from_files(list_of_corpuses)
        tokens = gen_tokens(lines)
        trigrams = gen_trigrams(tokens)

        bi, tri = defaultdict(int), defaultdict(int)

        for t0, t1, t2 in trigrams:
            bi[t0, t1] += 1
            tri[t0, t1, t2] += 1

        model = {}
        for (t0, t1, t2), freq in tri.items():
            if (t0, t1) in model:
                model[t0, t1].append((t2, freq / bi[t0, t1]))
            else:
                model[t0, t1] = [(t2, freq / bi[t0, t1])]
        return model

    @staticmethod
    def generate(model, length, prefix='-1'):
        phrase = ''
        bigrams = tuple(filter(lambda key: key[0] == prefix, model))
        # Если ввели 2 слова
        if tuple(prefix.split()) in model:
            t0, t1 = prefix.split()

        # Если ввели одно слово и оно есть в биграммах
        elif prefix != '-1' and len(bigrams) != 0:
            t0, t1 = choice(tuple(filter(lambda key: key[0] == prefix, model)))

        # Введеного слова нет в биграммах или ничего не ввели
        else:
            t0, t1 = choice(tuple(model))
        phrase += t0
        for i in range(length - len(prefix.split())):
            phrase += ' ' + t1
            t0, t1 = t1, choice(model[t0, t1])[0]
        return phrase.capitalize() + '.'

In [15]:
my_m = MyTrigramModel.fit(list_of_corpuses)

In [16]:
print(MyTrigramModel.generate(my_m, 30, 'я знаю'))

Я знаю своего отца наконец он встал мадемуазель де сен мерану несчастный дантес был брошен в море я рыл могилу как хоронил младенца он мог выйти из подчинения чумному.
